In [2]:
import re
import pandas as pd
import numpy as np
import requests
from lxml import html, etree
import time
import json
import datetime

PROXIES = {
 ## 'http': 'http://VPN4726:@ar.finevpn.org',
 ## 'https': 'https://VPN4726:@ar.finevpn.org',
}


In [5]:
## http://docs.python-guide.org/en/latest/scenarios/scrape/
##

class news_coindesk_parser():
    def __init__(self):
        ## self.link = 'https://www.coindesk.com/page/'
        self.save_dir = "news_coindesk/"
    
    def read_article_text(self,article_href, verbose=False):
        article_page = requests.get(article_href, proxies=PROXIES)
        article_html = html.fromstring(article_page.content)
        
        try:
            article_text = article_html.xpath('//div[@class="single-content"]')[0]
            text = etree.tostring(article_text).decode("utf-8") 
        except:
            print ("error reading text from page: ",article_href)
            text = ""

        text = re.sub("<em>The leader in blockchain news.*?</em>","",text)
        if verbose: print (text)
        return text
    
    def clean_text (self,article_text):
        article_text = re.sub("<script>.*?</script>","",article_text)
        article_text = re.sub("<.*?>","",article_text)
        article_text = re.sub("&#8217;","'",article_text)
        article_text = re.sub("&#[0-9]+;|\n"," ",article_text)
        article_text = re.sub(" {1,}"," ",article_text)

        return article_text

    def parse_page(self,page_num, verbose = True):

        page_link = "https://www.coindesk.com/page/" + str(page_num) + "/"
        r = requests.get(page_link, proxies = PROXIES)
        
        articles_html = html.fromstring(r.content)
        articles_html = articles_html.xpath('//div[@id="content"]')[0]
        articles_html = articles_html.xpath('.//div[@class="article medium bordered"]')

        if len(articles_html) == 0:
            print ("error reading page ",page_link)

        for i_article in articles_html:

            ## parsing date
            ##
            art_time = i_article.xpath('.//time/@datetime')[0]
            art_date = art_time[0:10]    

            art_title = i_article.xpath('.//div[@class="post-info"]/h3')[0]
            art_title = etree.tostring(art_title).decode('utf-8')
            art_title = self.clean_text(art_title)
            
            try:
                art_cont = i_article.xpath('.//div[@class="post-info"]/p')[1]
                art_cont = etree.tostring(art_cont).decode('utf-8')
                art_cont = self.clean_text(art_cont)
            except:
                art_cont = ""
                
            art_href =  i_article.xpath('.//div[@class="post-info"]/h3/a/@href')[0]

            if verbose: print (art_date,art_time, art_title, art_cont, art_href)

            ## read article text
            article_text = self.read_article_text(art_href,verbose=False)
            article_text = self.clean_text(article_text)

            
            ## saving article
            fn = re.sub("[^a-zA-Z0-9 ]","",art_title[0:20])
            file_name = self.save_dir + art_date + "_" + 'p_' + '{num:03d}'.format(num=page_num) + "_" + fn + ".txt"
            if verbose: print ('FILE NAME: ',file_name)
            f = open(file_name,'w', encoding = 'utf-8')
            f.write(art_title + "\n")
            f.write(art_time + "\n")
            f.write("<brief>" + art_cont + "</brief>\n")
            f.write(article_text)
            f.close()
        print ("page " + str(page_num) + " parsed")



            
bc_parser = news_coindesk_parser()

for i in range(25,26):
    bc_parser.parse_page(i,verbose=False)

error reading text from page:  https://www.coindesk.com/video-game-giant-unity-work-mobile-app-kiks-ico-token/
page 25 parsed
